### The Agenda of this notebook is just to deep dive into the data, perform some EDA to learn some patterns and then these points will help us in Feature Engineering

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
url = 'https://drive.google.com/file/d/1GRRfOnM0cwB9cRlyUVZHw0com64VFz9o/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url, encoding='iso-8859-1', on_bad_lines='skip')

In [18]:
df = df[df['Retailer Name'].isin(['LONDON DRUGS', 'JEAN-COUTU'])]

In [20]:
df

,Retailer Code,Retailer Name,Door Code,Door Name,Door Zip Code,Year,Week,UPC,SKU Category,SKU Descreption,POS Units,POS Amount
144,LON,LONDON DRUGS,216861,LONDON DRUGS #60,T1B 1E2,2022,152022,8435137727100,NaN,PRADA CANDY EDP V30ML V0,1.0,85.00
145,LON,LONDON DRUGS,239308,LONDON DRUGS #86,S4W 0B7,2022,152022,3360372058861,NaN,GA ADGH EDT SP 50ML SUB,1.0,95.00
146,LON,LONDON DRUGS,239308,LONDON DRUGS #86,S4W 0B7,2022,152022,3365440375055,NaN,YSL NUIT EDT SPRAY 60ML SUB,1.0,96.00
147,LON,LONDON DRUGS,221758,LONDON DRUGS #65,S7K 5S6,2022,152022,3360372058861,NaN,GA ADGH EDT SP 50ML SUB,1.0,95.00
148,LON,LONDON DRUGS,213529,LONDON DRUGS #09,V3R 1B9,2022,152022,3605970326494,NaN,LANC RML CR NUIT J75ML SUB,1.0,148.00
...,...,...,...,...,...,...,...,...,...,...,...,...
301915,LON,LONDON DRUGS,216828,LONDON DRUGS #24,T4R 3A2,2022,152022,3605970002534,NaN,LANC BMV NIGHT J50ML SUB,1.0,74.00
301916,LON,LONDON DRUGS,213529,LONDON DRUGS #09,V3R 1B9,2022,152022,772192624325,NaN,BIOTH BT RA SET WINTER 21,1.0,86.25
301917,LON,LONDON DRUGS,217762,LONDON DRUGS #29,V8V 3M4,2022,152022,3605970002534,NaN,LANC BMV NIGHT J50ML SUB,1.0,74.00
301918,LON,LONDON DRUGS,212731,LONDON DRUGS #44,V7T 1H9,2022,152022,3605970002534,NaN,LANC BMV NIGHT J50ML SUB,1.0,74.00


### Model to find the products allocation for each Retailer per Zip Code

### Product Allocation for London Drugs

In [25]:
df = df.rename(columns={'Door Zip Code': 'ZipCode', 'Retailer Code': 'RetailerCode', 'POS Units': 'Units'})

In [26]:
model_df_ld = df.loc[df['Retailer Name'] == 'LONDON DRUGS'][['RetailerCode', 'ZipCode', 'UPC', 'Units']]
model_df_ld.head()

,RetailerCode,ZipCode,UPC,Units
144,LON,T1B 1E2,8435137727100,1.0
145,LON,S4W 0B7,3360372058861,1.0
146,LON,S4W 0B7,3365440375055,1.0
147,LON,S7K 5S6,3360372058861,1.0
148,LON,V3R 1B9,3605970326494,1.0


In [27]:
unique_upcs = model_df_ld['UPC'].unique()
unique_upcs.shape

(801,)

In [29]:
model_df_ld_allocated = model_df_ld[model_df_ld['Units']>0]
model_df_ld_returned = model_df_ld[model_df_ld['Units']<0]

### Prepare the dataset where find the appropriate number of items against each upc for each retailers each Zip code i.e., UPC = items allocated - items returned for each retailer at each UPC i.e UPC 123 is being allocated 10 times to London Drugs and 13 times to Jean-Coutu and returned 7 times from London drugs and 5 times from Jean-Coutu so the desired data format would be:

London Drugs at L1k 0h5 = 10-7 = 3
London Drugs at M8W 1Y7 = 3-4 = -1

Jean Coutu at L1k 0h5 = 4-7 = -3
Jean Coutu at M8W 1Y7 = 10-2 = 8

In [30]:
model_df_ld_allocated

,RetailerCode,ZipCode,UPC,Units
144,LON,T1B 1E2,8435137727100,1.0
145,LON,S4W 0B7,3360372058861,1.0
146,LON,S4W 0B7,3365440375055,1.0
147,LON,S7K 5S6,3360372058861,1.0
148,LON,V3R 1B9,3605970326494,1.0
...,...,...,...,...
301915,LON,T4R 3A2,3605970002534,1.0
301916,LON,V3R 1B9,772192624325,1.0
301917,LON,V8V 3M4,3605970002534,1.0
301918,LON,V7T 1H9,3605970002534,1.0


In [32]:
unique_zipcodes = model_df_ld_allocated['ZipCode'].unique()

In [33]:
unique_zipcodes

array(['T1B 1E2', 'S4W 0B7', 'S7K 5S6', 'V3R 1B9', 'V4A 2H9', 'V7M 2K5',
       'V5J 0A2', 'V4K 1W4', 'V6K 2E3', 'V6M 2B4', 'V2Y 1P3', 'V3L 5H1',
       'V7A 5J3', 'V6Z 1E4', 'V4C 6P5', 'V2A 6W6', 'V2R 1A1', 'T6H 0K5',
       'V0N 1V0', 'T3E 7C4', 'V2L 3X3', 'T5K 2T6', 'V6X 2E3', 'V2C 1Y3',
       'V9A 7C5', 'T2Z 4J2', 'V5Y 0E4', 'T5E 6C2', 'T8N 3K8', 'V5P 3W2',
       'V9T 4K6', 'V3R 2W5', 'V3J 1N4', 'T5T 4M2', 'V7A 4X5', 'V5R 5L1',
       'V3B 5R5', 'V6E 1B5', 'T4R 3A2', 'V4T 1Y2', 'T8V 7L6', 'V9B 1V8',
       'V6S 2G6', 'V7T 1H9', 'V8V 3M4', 'V1Y 8J8', 'S7H 0V4', 'V2X 2V5',
       'T1Y 6M6', 'V2T 4M5', 'V5C 3Z6', 'T2L 1K8', 'V2V 6M7', 'V3S 2N6',
       'V5Z 1E6', 'T3K 2A8', 'V6B 0G6', 'T8A 4N5', 'V9L 3P5', 'V2T 0C5',
       'V3S 1Z2', 'T5P 4W1', 'T4B 3G3', 'R2M 5E5', 'V8X 1J8', 'V5H 2E2',
       'T6N 1K5', 'T1H 0E4', 'V1T 9H2', 'V9R 5E2', 'V9W 3A6', 'V8B 0G2',
       'V6G 1V9', 'V9N 2L8', 'V5K 1Z1', 'T2H 2N1', 'S4V 3A6', 'T3G 5T4'],
      dtype=object)

In [34]:
len(unique_zipcodes)

78

In [39]:
### UPCs for each zip code
ld_adjusted_products = {}
for zipcode in unique_zipcodes:
    print(zipcode)
    #get the sum of all the products that were allocated for each upc
    allocated_units = model_df_ld_allocated[model_df_ld_allocated['ZipCode'] == zipcode][['UPC', 'Units']]
    print(allocated_units)
    break
    
    
#     returned_units = model_df_ld_returned[model_df_ld_returned['UPC'] == zipcode]['UPC, Units']
#     ld_adjusted_products[upc] = allocated_units - returned_units

T1B 1E2
                  UPC  Units
144     8435137727100    1.0
19301   3614271844804    1.0
26261   3360377016132    1.0
37616   3614271256157    1.0
38229   3614273393492    1.0
38430   3360377034587    1.0
56596   3351500020812    1.0
56597   3605540496954    1.0
74321   8435137759811    1.0
80642   3614270201912    1.0
96534   3614272537385    2.0
112491  3614273568449    1.0
113993  3614273393539    1.0
114337  3605540886304    1.0
114338  3614270870026    1.0
131690  3360377022911    1.0
132953  3367729117264    2.0
133329  3605972455338    2.0
146073  3605540526712    1.0
150584  3614272050341    1.0
151972  3605540936634    1.0
152419  3614271256201    2.0
170862  3614272537439    1.0
170920  3367729586046    1.0
178086  3614272537439    1.0
204032  3614272968554    1.0
223826  3614271633279    1.0
224210  3365440375079    1.0
244586  3614271716101    1.0
244587  3614273393546    1.0
244640  3360372058878    1.0
262872  3605540911518    1.0
265738  3614272688339    1.0
272393

In [9]:
ld_adjusted_products = {}
for upc in unique_upcs:
    #get the sum of all the products that were allocated for each upc
    allocated_units = sum(model_df_ld_allocated[model_df_ld_allocated['UPC'] == upc]['POS Units'])
    returned_units = sum(model_df_ld_returned[model_df_ld_returned['UPC'] == upc]['POS Units'])
    ld_adjusted_products[upc] = allocated_units - returned_units

In [10]:
len(list(ld_adjusted_products.keys()))

801

### Product Allocation for JEAN-COUTU

In [12]:
model_df_jc = df.loc[df['Retailer Name'] == 'JEAN-COUTU'][['Retailer Code', 'UPC', 'POS Units']]
model_df_jc.head()

,Retailer Code,UPC,POS Units
2473,PJC,3614270129681,1.0
2474,PJC,3614270129681,1.0
2475,PJC,3614270129681,1.0
2476,PJC,3614270129681,1.0
2477,PJC,3614270129681,1.0


In [13]:
unique_upcs = model_df_jc['UPC'].unique()
unique_upcs.shape

(1002,)

In [14]:
model_df_jc_allocated = model_df_jc[model_df_jc['POS Units']>0]
model_df_jc_returned = model_df_jc[model_df_jc['POS Units']<0]

### Prepare the dataset where find the appropriate number of items against each upc i.e., UPC = items allocated - items returned for each retailer i.e UPC 123 is being allocated 10 times to London Drugs and 13 times to Jean-Coutu and returned 7 times from London drugs and 5 times from Jean-Coutu so the desired data format would be:

London Drugs = 10-7 = 3
Jean Coutu = 13-5 = 8

In [15]:
len(unique_upcs)

1002

In [16]:
jc_adjusted_products = {}
for upc in unique_upcs:
    #get the sum of all the products that were allocated for each upc
    allocated_units = sum(model_df_jc_allocated[model_df_jc_allocated['UPC'] == upc]['POS Units'])
    returned_units = sum(model_df_jc_returned[model_df_jc_returned['UPC'] == upc]['POS Units'])
    jc_adjusted_products[upc] = allocated_units - returned_units

In [17]:
len(list(jc_adjusted_products.keys()))

1002

In [18]:
jc_upcs = len(jc_adjusted_products.keys())
ld_upcs = len(ld_adjusted_products.keys())

In [19]:
retailers = []
retailers.extend(['London Drugs' for x in range(ld_upcs)])
retailers.extend(['Jean Coutu' for x in range(jc_upcs)])
len(retailers)

1803

In [20]:
upcs_for_each_retailer = []
upcs_for_each_retailer.extend([x[0] for x in jc_adjusted_products.items()])
upcs_for_each_retailer.extend([x[0] for x in ld_adjusted_products.items()])
len(upcs_for_each_retailer)

1803

In [21]:
adjusted_upc_amount_for_each_retailer = []
adjusted_upc_amount_for_each_retailer.extend([x[1] for x in jc_adjusted_products.items()])
adjusted_upc_amount_for_each_retailer.extend([x[1] for x in ld_adjusted_products.items()])
len(adjusted_upc_amount_for_each_retailer)

1803

In [22]:
product_allocation_for_each_retailer = pd.DataFrame({'Retailer':retailers, 
                                                   'UPC': upcs_for_each_retailer,
                                                   'Unit': adjusted_upc_amount_for_each_retailer})

In [23]:
product_allocation_for_each_retailer.tail()

,Retailer,UPC,Unit
1798,Jean Coutu,3614272889491,15.0
1799,Jean Coutu,3614270256332,2.0
1800,Jean Coutu,772191735237,3.0
1801,Jean Coutu,3614271412591,2.0
1802,Jean Coutu,3605970761134,1.0


In [24]:
reatail_dict = {'London Drugs':0, 'Jean Coutu': 1}

In [25]:
product_allocation_for_each_retailer['retailer'] = product_allocation_for_each_retailer['Retailer'].map(reatail_dict)

In [26]:
product_allocation_for_each_retailer.drop(['Retailer'], axis=1, inplace=True)

In [27]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [28]:
X = product_allocation_for_each_retailer.drop(['Unit'], axis=1)
y = product_allocation_for_each_retailer['Unit']

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [30]:
model = RandomForestRegressor()

In [31]:
model.fit(X_train, y_train)

RandomForestRegressor()

In [32]:
preds = model.predict(X_test)

In [33]:
from sklearn.metrics import r2_score
r2_score(y_test, preds)

0.09630145888953856

### UPCS with 0 allocation

In [34]:
for d in jc_adjusted_products.items():
    if d[1] == 0.0:
        print(d[0])